# Locating street markets: Analysis from qualitative and quantitative factors using a discrete choice model and Fuzzy AHP

## Fuzzy AHP for Criteria

In [2]:
# Instalar Librería F-AHP
!pip install pyDecision

  Using cached pyDecision-4.5.8-py3-none-any.whl.metadata (21 kB)
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.9 kB 262.6 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.9 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 296.2 kB/s eta 0:00:00
  Using cached llmx-0.0.21a0-py3-none-any.whl.metadata (8.9 kB)
  Using cached openai-1.37.1-py3-none-any.whl.metadata (22 kB)
  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.5.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (12

In [3]:
# Required Libraries
import pyDecision
import numpy as np
import pandas as pd
import sys
import math
np.set_printoptions(threshold=sys.maxsize)
# Fuzzy AHP
from pyDecision.algorithm import fuzzy_ahp_method

In [4]:
#RI Modified for the matriz size 50

def fuzzy_ahp_method(dataset):
    row_sum = []
    s_row   = []
    f_w     = []
    d_w     = []
    inc_rat = np.array([0, 0, 0, 0.52, 0.89, 1.13, 1.25, 1.35, 1.43, 1.47, 1.5, 1.53, 1.54, 1.56, 1.57, 1.59, 1.6, 1.61, 1.61, 1.62, 1.63, 1.63, 1.64, 1.65, 1.65, 1.66, 1.66, 1.66, 1.67, 1.67, 1.67, 1.67, 1.68, 1.68, 1.68, 1.68, 1.69, 1.69, 1.69, 1.69, 1.69, 1.70, 1.70, 1.70, 1.70, 1.70, 1.70, 1.70, 1.70, 1.71, 1.71])
    X       = [(item[0] + 4*item[1] + item[2])/6 for i in range(0, len(dataset)) for item in dataset[i] ]
    X       = np.asarray(X)
    X       = np.reshape(X, (len(dataset), len(dataset)))
    for i in range(0, len(dataset)):
        a, b, c = 1, 1, 1
        for j in range(0, len(dataset[i])):
            d, e, f = dataset[i][j]
            a, b, c = a*d, b*e, c*f
        row_sum.append( (a, b, c) )
    L, M, U = 0, 0, 0
    for i in range(0, len(row_sum)):
        a, b, c = row_sum[i]
        a, b, c = a**(1/len(dataset)), b**(1/len(dataset)), c**(1/len(dataset))
        s_row.append( ( a, b, c ) )
        L = L + a
        M = M + b
        U = U + c
    for i in range(0, len(s_row)):
        a, b, c = s_row[i]
        a, b, c = a*(U**-1), b*(M**-1), c*(L**-1)
        f_w.append( ( a, b, c ) )
        d_w.append( (a + b + c)/3 )
    n_w      = [item/sum(d_w) for item in d_w]
    vector   = np.sum(X*n_w, axis = 1)/n_w
    lamb_max = np.mean(vector)
    cons_ind = (lamb_max - X.shape[1])/(X.shape[1] - 1)
    rc       = cons_ind/inc_rat[X.shape[1]]
    return f_w, d_w, n_w, rc


In [5]:
# Dataset for Criterias
dataset = list([
    [ (  1,   1,   1), (  6,   7,   8), (  4,   5,   6), (  9,   9,   9), (  4,   5,   6), (  6,   7,   8), (  1,   1,   1), (  4,   5,   6), (  2,   3,   4), (  4,   5,   6) ], #Pop Density
    [ (1/8, 1/7, 1/6), (  1,   1,   1), (1/4, 1/3, 1/2), (  2,   3,   4), (1/4, 1/3, 1/2), (  1,   1,   1), (1/8, 1/7, 1/6), (1/4, 1/3, 1/2), (1/6, 1/5, 1/4), (1/4, 1/3, 1/2) ], #Stratum
    [ (1/6, 1/5, 1/4), (  2,   3,   4), (  1,   1,   1), (  4,   5,   6), (  1,   1,   1), (  2,   3,   4), (1/6, 1/5, 1/4), (  1,   1,   1), (1/4, 1/3, 1/2), (   1,  1,   1) ], #Roads
    [ (1/9, 1/9, 1/9), (1/4, 1/3, 1/2), (1/6, 1/5, 1/4), (  1,   1,   1), (1/6, 1/5, 1/4), (1/4, 1/3, 1/2), (1/9, 1/9, 1/9), (1/6, 1/5, 1/4), (1/8, 1/7, 1/6), (1/6, 1/5, 1/4) ], #Sidewalks
    [ (1/6, 1/5, 1/4), (  2,   3,   4), (  1,   1,   1), (  4,   5,   6), (  1,   1,   1), (  2,   3,   4), (1/6, 1/5, 1/4), (  1,   1,   1), (1/4, 1/3, 1/2), (  1,   1,   1) ], #Morphology
    [ (1/8, 1/7, 1/6), (  1,   1,   1), (1/4, 1/3, 1/2), (  2,   3,   4), (1/4, 1/3, 1/2), (  1,   1,   1), (1/8, 1/7, 1/6), (1/4, 1/3, 1/4), (1/6, 1/5, 1/4), (1/4, 1/3, 1/2) ], #Public Services Desnity
    [ (  1,   1,   1), (  6,   7,   8), (  4,   5,   6), (  9,   9,   9), (  4,   5,   6), (  6,   7,   8), (  1,   1,   1), (  4,   5,   6), (  2,   3,   4), (  4,   5,   6) ], #Land Use
    [ (1/6, 1/5, 1/4), (  2,   3,   4), (  1,   1,   1), (  4,   5,   6), (  1,   1,   1), (1/4, 1/3, 1/2), (1/6, 1/5, 1/4), (  1,   1,   1), (1/4, 1/3, 1/2), (  1,   1,   1) ], #Traffic
    [ (1/4, 1/3, 1/2), (  4,   5,   6), (  2,   3,   4), (  6,   7,   8), (  2,   3,   4), (  4,   5,   6), (1/4, 1/3, 1/2), (  2,   3,   4), (  1,   1,   1), (  2,   3,   4) ], #Transport
    [ (1/6, 1/5, 1/4), (  2,   3,   4), (  1,   1,   1), (  4,   5,   6), (  1,   1,   1), (  2,   3,   4), (1/6, 1/5, 1/4), (  2,   3,   4), (1/4, 1/3, 1/2), (  1,   1,   1) ], #Housing
    ])

In [6]:
# Call Fuzzy AHP Function
fuzzy_weights, defuzzified_weights, normalized_weights, rc = fuzzy_ahp_method(dataset)

In [7]:
# Fuzzy Weigths for Criterias
for i in range(0, len(fuzzy_weights)):
    print('g'+str(i+1)+': ', np.around(fuzzy_weights[i], 3))

g1:  [0.193 0.266 0.363]
g2:  [0.02  0.028 0.043]
g3:  [0.046 0.065 0.091]
g4:  [0.011 0.015 0.022]
g5:  [0.046 0.065 0.091]
g6:  [0.02  0.028 0.04 ]
g7:  [0.193 0.266 0.363]
g8:  [0.038 0.052 0.074]
g9:  [0.091 0.142 0.218]
g10:  [0.05  0.072 0.105]


In [8]:
# Crisp Weigths for Criterias
for i in range(0, len(defuzzified_weights)):
    print('g'+str(i+1)+': ', round(defuzzified_weights[i], 3))

g1:  0.274
g2:  0.03
g3:  0.068
g4:  0.016
g5:  0.068
g6:  0.029
g7:  0.274
g8:  0.055
g9:  0.151
g10:  0.076


In [9]:
# Normalized Weigths for Criterias
for i in range(0, len(normalized_weights)):
    print('g'+str(i+1)+': ', round(normalized_weights[i], 3))

criteria_normalized_weights= normalized_weights.copy()

g1:  0.263
g2:  0.029
g3:  0.065
g4:  0.016
g5:  0.065
g6:  0.028
g7:  0.263
g8:  0.053
g9:  0.145
g10:  0.073


In [10]:
# Consistency Ratio for Criterias
if ( rc >0.10):
    print('The solution is inconsistent, the pairwise comparisons must be reviewed')
else:
    print('The solution is consistent')

print('RC: ' + str(round(rc, 2)))

The solution is consistent
RC: 0.04







## Fuzzy AHP for candidates






In [11]:
def populate_matrix(a):
    '''
    input upper square diagonal matrix
    output full criteria matrix
    '''
    n = a.shape[0]
    for i in range(n):
        for j in range(i,n):
            a[j,i] = 1/a[i,j]
    return a

In [12]:
# Read the table with the criteria
df = np.array(dataset, dtype = object)

In [13]:
#Criteria matrix
criteria_mat = df
criteria_mat

array([[[1, 1, 1],
        [6, 7, 8],
        [4, 5, 6],
        [9, 9, 9],
        [4, 5, 6],
        [6, 7, 8],
        [1, 1, 1],
        [4, 5, 6],
        [2, 3, 4],
        [4, 5, 6]],

       [[0.125, 0.14285714285714285, 0.16666666666666666],
        [1, 1, 1],
        [0.25, 0.3333333333333333, 0.5],
        [2, 3, 4],
        [0.25, 0.3333333333333333, 0.5],
        [1, 1, 1],
        [0.125, 0.14285714285714285, 0.16666666666666666],
        [0.25, 0.3333333333333333, 0.5],
        [0.16666666666666666, 0.2, 0.25],
        [0.25, 0.3333333333333333, 0.5]],

       [[0.16666666666666666, 0.2, 0.25],
        [2, 3, 4],
        [1, 1, 1],
        [4, 5, 6],
        [1, 1, 1],
        [2, 3, 4],
        [0.16666666666666666, 0.2, 0.25],
        [1, 1, 1],
        [0.25, 0.3333333333333333, 0.5],
        [1, 1, 1]],

       [[0.1111111111111111, 0.1111111111111111, 0.1111111111111111],
        [0.25, 0.3333333333333333, 0.5],
        [0.16666666666666666, 0.2, 0.25],
        [1, 

In [22]:
#Lee la tabla de calificaciones
df_candidates = pd.read_excel('Tabla_Resumen_Bosa_FAHP.xlsx')
df_candidates.set_index('ID_PARQUE', inplace=True)
df_candidates = df_candidates.iloc[:50,:]
df_candidates.head(10)

df_candidates['Pop Density'].fillna(int(df_candidates['Pop Density'].mean()), inplace=True)

C:\Users\jaime\AppData\Local\Temp\ipykernel_25328\4071304207.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_candidates['Pop Density'].fillna(int(df_candidates['Pop Density'].mean()), inplace=True)


In [24]:
candidate_matrix = df_candidates.iloc[:,0:10].to_numpy()
candidate_matrix.shape
print(candidate_matrix)

[[12.6  2.   0.   0.   0.   0.   0.   0.   1.   3. ]
 [ 7.4  2.   0.   0.   1.   0.   0.   0.   0.   1. ]
 [ 8.9  2.   1.   1.   1.   1.   0.   0.   1.   3. ]
 [15.   2.   0.   1.   1.   0.   0.   0.   0.   1. ]
 [ 4.9  2.   0.   1.   1.   1.   0.   0.   1.   3. ]
 [ 7.8  2.   1.   1.   1.   0.   0.   0.   1.   3. ]
 [ 6.9  2.   1.   1.   1.   1.   0.  -2.   1.   0. ]
 [ 8.   1.   1.   1.   1.   0.   0.   0.   0.   2. ]
 [ 8.6  2.   0.   0.   0.   0.   0.   0.   0.   2. ]
 [11.1  2.   1.   1.   1.   1.   0.  -1.   1.   1. ]
 [ 6.3  2.   0.   1.   1.   1.   0.  -2.   1.   1. ]
 [10.2  2.   1.   0.   0.   1.   0.   0.   1.   1. ]
 [ 6.9  2.   1.   1.   1.   1.  -1.   0.   1.   1. ]
 [ 3.8  2.   1.   1.   0.   0.  -1.  -1.   1.   2. ]
 [11.2  2.   1.   0.   0.   0.  -1.  -1.   1.   1. ]
 [ 7.4  2.   0.   1.   1.   0.   0.   0.   0.   1. ]
 [ 8.5  2.   0.   0.   0.   0.   0.   0.   0.   1. ]
 [ 6.8  2.   0.   0.   1.   0.   0.   0.   0.   1. ]
 [ 8.3  2.   1.   0.   0.   1.  -1.   0.   1. 

In [25]:
def set_factor_matrix (candidate_matrix, col):
    '''
    sets the factor matrix
    '''
    n = candidate_matrix.shape[0]
    factor_matrix = np.zeros((n,n))
    NumMat1 = 13.1/9
    for i in range(n):
        for k in range(i,n):
            if col == 0:
                if candidate_matrix[i,col] == candidate_matrix[k,col]:
                    factor_matrix[i,k] = 1
                elif candidate_matrix[i,col] > candidate_matrix[k,col]:
                    factor_matrix[i,k] = math.ceil((candidate_matrix[i,col] - candidate_matrix[k,col])/NumMat1)
                else:
                    factor_matrix[i,k] =  1/abs(math.floor((candidate_matrix[i,col] - candidate_matrix[k,col])/NumMat1))
            elif col ==1 or col ==7:
                if candidate_matrix[i,col] == candidate_matrix[k,col]:
                    factor_matrix[i,k] = 1
                elif candidate_matrix[i,col] < candidate_matrix[k,col]:
                    if candidate_matrix[k,col] - candidate_matrix[i,col] ==2:
                        factor_matrix[i,k] = 5
                    else:
                        factor_matrix[i,k] = 3
                elif candidate_matrix[i,col] > candidate_matrix[k,col]:
                    if candidate_matrix[i,col] - candidate_matrix[k,col] ==2:
                        factor_matrix[i,k] = 1/5
                    else:
                        factor_matrix[i,k] = 1/3
            elif col ==9:
                if candidate_matrix[i,col] == candidate_matrix[k,col]:
                    factor_matrix[i,k] = 1
                elif candidate_matrix[i,col] > candidate_matrix[k,col]:
                    if candidate_matrix[i,col] - candidate_matrix[k,col] ==3:
                        factor_matrix[i,k] = 7
                    else:
                        if candidate_matrix[i,col] - candidate_matrix[k,col] ==2:
                            factor_matrix[i,k] = 5
                        else:
                            factor_matrix[i,k] = 3
                elif candidate_matrix[i,col] < candidate_matrix[k,col]:
                    if candidate_matrix[k,col] - candidate_matrix[i,col] ==3:
                        factor_matrix[i,k] = 1/7
                    else:
                        if candidate_matrix[k,col] - candidate_matrix[i,col] ==2:
                            factor_matrix[i,k] = 1/5
                        else:
                            factor_matrix[i,k] = 1/3
            else:
                if candidate_matrix[i,col] == candidate_matrix[k,col]:
                    factor_matrix[i,k] = 1
                elif candidate_matrix[i,col] > candidate_matrix[k,col]:
                    factor_matrix[i,k] = 5
                else:
                    factor_matrix[i,k] = 1/5
    factor_matrix = populate_matrix(factor_matrix)
    return factor_matrix

In [26]:
#Calcula las matrices para todas las opciones y todos los criterios
#Crea una lista de matrices numpy
n = criteria_mat.shape[0] #number of cols of criteria matrix
factor_matrices = []
crs = []
for col in range (n):
    factor_m =  set_factor_matrix(candidate_matrix, col)
    factor_matrices.append(factor_m)
print(factor_matrices)

[array([[1.        , 4.        , 3.        , 0.5       , 6.        ,
        4.        , 4.        , 4.        , 3.        , 2.        ,
        5.        , 2.        , 4.        , 7.        , 1.        ,
        4.        , 3.        , 4.        , 3.        , 1.        ,
        2.        , 3.        , 4.        , 1.        , 4.        ],
       [0.25      , 1.        , 0.5       , 0.16666667, 2.        ,
        1.        , 1.        , 1.        , 1.        , 0.33333333,
        1.        , 0.5       , 1.        , 3.        , 0.33333333,
        1.        , 1.        , 1.        , 1.        , 0.25      ,
        0.33333333, 0.5       , 1.        , 0.2       , 1.        ],
       [0.33333333, 2.        , 1.        , 0.2       , 3.        ,
        1.        , 2.        , 1.        , 1.        , 0.5       ,
        2.        , 1.        , 2.        , 4.        , 0.5       ,
        2.        , 1.        , 2.        , 1.        , 0.33333333,
        0.5       , 1.        , 2.        , 0

In [28]:
factor_matrices = np.array(factor_matrices, dtype=object)
for i in range(10):
    for j in range(25):
        for k in range(25):
            if np.all(factor_matrices[i][j][k] == 1):
                factor_matrices[i][j][k] = (  1,   1,   1)
            elif np.all(factor_matrices[i][j][k] == 2):
                 factor_matrices[i][j][k] = (  1,   2,   3)
            elif np.all(factor_matrices[i][j][k] == 3):
                 factor_matrices[i][j][k] = (  2,   3,   4)
            elif np.all(factor_matrices[i][j][k] == 4):
                 factor_matrices[i][j][k] = (  3,   4,   5)
            elif np.all(factor_matrices[i][j][k] == 5):
                 factor_matrices[i][j][k] = (  4,   5,   6)
            elif np.all(factor_matrices[i][j][k] == 6):
                 factor_matrices[i][j][k] = (  5,   6,   7)
            elif np.all(factor_matrices[i][j][k] == 7):
                 factor_matrices[i][j][k] = (  6,   7,   8)
            elif np.all(factor_matrices[i][j][k] == 8):
                 factor_matrices[i][j][k] = (  7,   8,   9)
            elif np.all(factor_matrices[i][j][k] == 9):
                 factor_matrices[i][j][k] = (  9,   9,   9)


            elif np.all(factor_matrices[i][j][k] == 1/2):
                 factor_matrices[i][j][k] = (  1/3,   1/2,   1)
            elif np.all(factor_matrices[i][j][k] == 1/3):
                 factor_matrices[i][j][k] = (  1/4,   1/3,   1/2)
            elif np.all(factor_matrices[i][j][k] == 1/4):
                 factor_matrices[i][j][k] = (  1/5,   1/4,   1/3)
            elif np.all(factor_matrices[i][j][k] == 1/5):
                 factor_matrices[i][j][k] = (  1/6,   1/5,   1/4)
            elif np.all(factor_matrices[i][j][k] == 1/6):
                 factor_matrices[i][j][k] = (  1/7,   1/6,   1/5)
            elif np.all(factor_matrices[i][j][k] == 1/7):
                 factor_matrices[i][j][k] = (  1/8,   1/7,   1/6)
            elif np.all(factor_matrices[i][j][k] == 1/8):
                 factor_matrices[i][j][k] = (  1/9,   1/8,   1/7)
            elif np.all(factor_matrices[i][j][k] == 1/9):
                 factor_matrices[i][j][k] = (  1/9,   1/9,   1/9)

            else:
                factor_matrices[i][j][k] = (  0.1,   0.2,   0.3)

In [29]:
# View full matrix
np.set_printoptions(threshold=sys.maxsize)
factor_matrices

array([[[(0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
         (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
         (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
         (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
         (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
         (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
         (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
         (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
         (0.1, 0.2, 0.3)],
        [(0.2, 0.25, 0.3333333333333333), (1, 1, 1),
         (0.3333333333333333, 0.5, 1),
         (0.14285714285714285, 0.16666666666666666, 0.2), (1, 2, 3),
         (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1),
         (0.25, 0.3333333333333333, 0.5), (1, 1, 1),
         (0.3333333333333333, 0.5, 1), (1, 1, 1), (2, 3, 4),
         (0.25, 0.3333333333333333, 0.5), (1, 1, 1), (1, 1, 1),
         (1, 1, 1), (1, 1, 1), (0.2, 0.25, 0.3333333333333333),
         (0.25, 0.3333333333333333, 0

In [30]:
# View matrix shape
np.shape(factor_matrices)

(10, 25, 25)

In [31]:
factor_matrices[0]

array([[(0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
        (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
        (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
        (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
        (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
        (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
        (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
        (0.1, 0.2, 0.3), (0.1, 0.2, 0.3), (0.1, 0.2, 0.3),
        (0.1, 0.2, 0.3)],
       [(0.2, 0.25, 0.3333333333333333), (1, 1, 1),
        (0.3333333333333333, 0.5, 1),
        (0.14285714285714285, 0.16666666666666666, 0.2), (1, 2, 3),
        (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1),
        (0.25, 0.3333333333333333, 0.5), (1, 1, 1),
        (0.3333333333333333, 0.5, 1), (1, 1, 1), (2, 3, 4),
        (0.25, 0.3333333333333333, 0.5), (1, 1, 1), (1, 1, 1), (1, 1, 1),
        (1, 1, 1), (0.2, 0.25, 0.3333333333333333),
        (0.25, 0.3333333333333333, 0.5), (0.3333333333

#### START FOR EACH MATRIX (To develop by A & V)

In [38]:
def fuzzy_alternatives(factor_matrices):
    matrices_50x50 = []
    resultados = pd.DataFrame()
    ratio=[]

    for l in range(10):
        matriz_A=[]
        for i in range(l+1):
      # Tomar la "capa" o "matriz" de (50, 50) para la iteración actual
           matriz_A = factor_matrices[i, :, :]

      # Agregar la matriz (50, 50) a la lista
           matrices_50x50.append(matrices_50x50)

    dataset_A = list(matriz_A)
    fuzzy_weights, defuzzified_weights, normalized_weights, rc = fuzzy_ahp_method(dataset_A)
    resultados['Criterio '+ str(l+1)] = normalized_weights
    ratio.append(rc)

    return resultados,ratio

In [39]:
alternatives,rc=fuzzy_alternatives(factor_matrices)
rc

[np.float64(0.011891134670478)]

In [40]:
def final_weight(alternatives, criteria):
    alternatives=alternatives.to_numpy()
    result=[]
    for i in range (25):
        multiply=[]
        sum = 0
        for j in range (10):
            multiply.append(alternatives[i,j] * criteria[j])

        for k in multiply:
            sum = sum + k

            result.append(sum)

    return result

In [41]:

# Verificar dimensiones de las variables
print('Dimensiones de alternatives:', alternatives.shape)
print('Dimensiones de criteria_normalized_weights:', len(criteria_normalized_weights))

result=final_weight(alternatives,criteria_normalized_weights)
result

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
result = pd.DataFrame(result)
result.to_excel('Resultados.xlsx')